<a href="https://colab.research.google.com/github/AarishB/FragrAI/blob/main/FragrAIi_pynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import OneHotEncoder
import scipy.sparse as sp
import ast
from collections import Counter

# Load the perfume dataset
df = pd.read_csv('perfumes_table.csv')

# Clean and preprocess the data
df['notes'] = df['notes'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) and x.startswith('[') else [])

# 🔥 WEIGHTED KEYWORDS - Boost important scent descriptors 3x
scent_descriptors = {
    'woody', 'fresh', 'floral', 'spicy', 'citrus', 'sweet', 'warm', 'cool',
    'aromatic', 'oriental', 'aquatic', 'marine', 'green', 'fruity', 'powdery',
    'amber', 'musky', 'earthy', 'smoky', 'leathery', 'vanilla', 'creamy',
    'balsamic', 'resinous', 'herbal', 'minty', 'animalic', 'aldehydic',
    'gourmand', 'ozonic', 'metallic', 'soapy', 'clean', 'dirty', 'sharp',
    'soft', 'rich', 'light', 'heavy', 'bright', 'dark', 'mysterious',
    'seductive', 'elegant', 'bold', 'subtle', 'intense', 'delicate',
    'silky', 'velvety', 'smooth', 'rough', 'dry', 'wet', 'airy', 'dense',
    'chypre', 'fougere', 'tobacco', 'incense', 'oud', 'leather', 'musk',
    'patchouli', 'vetiver', 'sandalwood', 'cedar', 'bergamot', 'lavender',
    'rose', 'jasmine', 'iris', 'neroli', 'tonka', 'cocoa'
}

def extract_weighted_keywords(description):
    """Extract keywords and repeat important scent descriptors 3x for TF-IDF boost"""
    if pd.isna(description):
        return []
    
    words = description.lower().split()
    weighted = []
    
    for word in words:
        weighted.append(word)  # Add once
        if word in scent_descriptors:
            weighted.extend([word, word])  # Add 2 more times (3x total)
    
    return weighted

# Create combined notes with weighted keywords
print("🔥 Applying weighted keyword extraction...")
df['description_keywords'] = df['description'].apply(extract_weighted_keywords)
df['combined_notes_plus'] = df.apply(lambda row: row['notes'] + row['description_keywords'], axis=1)
print(f"   ✓ Weighted {len(scent_descriptors)} scent descriptors 3x")

# Extract gender from title
def extract_gender(title):
    if pd.isna(title):
        return 'unisex'
    title_lower = str(title).lower()
    
    # Check for both genders first
    has_women = any(word in title_lower for word in ['women', 'woman', 'female', 'lady', 'femme'])
    has_men = any(word in title_lower for word in ['men', 'man', 'male', 'homme'])
    
    if has_women and has_men:
        return 'unisex'
    elif has_women:
        return 'feminine'
    elif has_men:
        return 'masculine'
    else:
        return 'unisex'

df['gender'] = df['title'].apply(extract_gender)

# Extract season (simplified - in real implementation you'd parse descriptions)
df['season'] = 'all'  # Default
season_keywords = {
    'spring': ['spring', 'fresh', 'floral', 'light'],
    'summer': ['summer', 'citrus', 'marine', 'fresh'],
    'fall': ['fall', 'autumn', 'warm', 'amber', 'spicy'],
    'winter': ['winter', 'warm', 'vanilla', 'oriental']
}

for season, keywords in season_keywords.items():
    mask = df['description'].fillna('').str.lower().apply(lambda x: any(kw in x for kw in keywords))
    df.loc[mask, 'season'] = season

# Gender filtering keywords
GENDER_KEYWORDS = {
    'feminine': ['women', 'lady', 'feminine', 'floral', 'sweet', 'powdery'],
    'masculine': ['men', 'masculine', 'woody', 'spicy', 'leather', 'tobacco']
}

print("\n✓ Dataset loaded successfully!")
print(f"   - Total perfumes: {len(df)}")
print(f"   - Columns: {list(df.columns)}")
print(f"   - Gender distribution: {df['gender'].value_counts().to_dict()}")
print(f"   - Season distribution: {df['season'].value_counts().to_dict()}")

# === DYNAMIC NOTE EXTRACTION FUNCTIONS ===

def extract_notes_for_keywords(df, keywords, top_n=25):
    """
    Extract top N notes from fragrances whose descriptions contain the keywords.
    
    Args:
        df: DataFrame with 'description' and 'notes' columns
        keywords: List of words to search for (e.g., ['bold', 'woody', 'intense'])
        top_n: Number of top notes to return
    
    Returns:
        List of most common notes in matching fragrances
    """
    # Filter fragrances with ANY of these keywords in description
    mask = df['description'].fillna('').str.lower().apply(
        lambda x: any(kw.lower() in x for kw in keywords)
    )
    matching_fragrances = df[mask]
    
    # Collect all notes from matching fragrances
    all_notes = []
    for notes_list in matching_fragrances['notes']:
        if isinstance(notes_list, list):
            all_notes.extend([n.lower().strip() for n in notes_list if n])
    
    # Count frequency
    note_counts = Counter(all_notes)
    
    # Return top N most common notes
    top_notes = [note for note, count in note_counts.most_common(top_n)]
    
    return top_notes


def build_dynamic_note_profile(seed_notes, keywords, df, top_n=25):
    """
    Hybrid approach: Start with seed notes, expand with dynamically extracted notes.
    
    Args:
        seed_notes: Your curated starting notes (e.g., ['oud', 'leather'])
        keywords: Description keywords to search for (e.g., ['bold', 'intense'])
        df: DataFrame
        top_n: Total number of notes to return
    
    Returns:
        Combined list of seed + discovered notes (deduplicated)
    """
    # Start with seed notes (lowercase)
    final_notes = [n.lower().strip() for n in seed_notes]
    
    # Extract notes from keyword-matching fragrances
    dynamic_notes = extract_notes_for_keywords(df, keywords, top_n=top_n*2)
    
    # Add dynamic notes (deduplicate)
    for note in dynamic_notes:
        if note not in final_notes:
            final_notes.append(note)
    
    # Cap at top_n
    return final_notes[:top_n]


print("\n✓ Dynamic note extraction functions loaded!")
print("   - extract_notes_for_keywords(): Find notes by description keywords")
print("   - build_dynamic_note_profile(): Hybrid seed + dynamic approach")

🔥 Applying weighted keyword extraction...
   ✓ Weighted 75 scent descriptors 3x

✓ Dataset loaded successfully!
   - Total perfumes: 84144
   - Columns: ['rating', 'notes', 'designer', 'reviews', 'description', 'url', 'title', 'description_keywords', 'combined_notes_plus', 'gender', 'season']
   - Gender distribution: {'unisex': 69759, 'masculine': 14385}
   - Season distribution: {'fall': 25602, 'winter': 19642, 'spring': 17281, 'all': 16912, 'summer': 4707}

✓ Dynamic note extraction functions loaded!
   - extract_notes_for_keywords(): Find notes by description keywords
   - build_dynamic_note_profile(): Hybrid seed + dynamic approach


In [5]:
# === STEP 5: Build TF-IDF vectors for cosine similarity ===
df['note_string'] = df['combined_notes_plus'].apply(lambda lst: ' '.join(lst))
tfidf = TfidfVectorizer(
    ngram_range=(1,2),  # ⚡ Include note pairs for better matching
    min_df=1,           # ⚡ Include rare notes (was 2)
    max_features=3000,  # ⚡ More vocabulary (was 2000)
    sublinear_tf=True   # ⚡ Better TF scaling for high similarity
)
X_tfidf = tfidf.fit_transform(df['note_string'])

# === STEP 6: Create season vectors ===
from sklearn.preprocessing import OneHotEncoder
season_encoder = OneHotEncoder(sparse_output=False)
season_vectors = season_encoder.fit_transform(df[['season']])
season_feature_names = season_encoder.get_feature_names_out(['season'])

print("✓ Data preprocessing complete!")
print(f"   - Dataset: {len(df)} perfumes")
print(f"   - TF-IDF features: {X_tfidf.shape[1]} dimensions (includes note pairs)")
print(f"   - Season features: {season_vectors.shape[1]} dimensions")
print(f"   - Total notes vocabulary: {len(tfidf.get_feature_names_out())} unique notes/pairs")

# === STEP 7: Combine note and season vectors ===
import scipy.sparse as sp
from sklearn.preprocessing import StandardScaler

# Convert season vectors to sparse and combine
X_season_sparse = sp.csr_matrix(season_vectors)

# Combine TF-IDF (notes) + Season vectors
# Weight: 85% notes, 15% seasons for similarity
note_weight = 0.85
season_weight = 0.15

X_combined = sp.hstack([
    X_tfidf * note_weight,
    X_season_sparse * season_weight
], format='csr')

print(f"   - Combined features: {X_combined.shape[1]} dimensions ({note_weight*100:.0f}% notes + {season_weight*100:.0f}% seasons)")

# === STEP 2: BUILD DYNAMIC NOTE MAPPINGS FROM DATASET ===
print("\n🔍 Building dynamic note mappings from dataset...")
print("=" * 60)

# === PERSONALITY MAPPINGS (Hybrid: Seed + Dynamic) ===
PERSONALITY_NOTES = {
    'Bold & Adventurous (confident, outgoing, takes charge)': build_dynamic_note_profile(
        seed_notes=['oud', 'leather', 'tobacco', 'pepper'],
        keywords=['bold', 'intense', 'strong', 'powerful', 'confident', 'woody', 'spicy', 'masculine'],
        df=df,
        top_n=25
    ),
    'Calm & Thoughtful (organized, reliable, introspective)': build_dynamic_note_profile(
        seed_notes=['lavender', 'tea', 'musk', 'sandalwood'],
        keywords=['calm', 'soft', 'gentle', 'subtle', 'peaceful', 'serene', 'light', 'clean'],
        df=df,
        top_n=25
    ),
    'Energetic & Social (enthusiastic, expressive, loves attention)': build_dynamic_note_profile(
        seed_notes=['citrus', 'bergamot', 'grapefruit', 'mint'],
        keywords=['fresh', 'energetic', 'vibrant', 'bright', 'lively', 'sparkling', 'citrus'],
        df=df,
        top_n=25
    ),
    'Creative & Open-minded (curious, artistic, unconventional)': build_dynamic_note_profile(
        seed_notes=['jasmine', 'incense', 'patchouli', 'fig'],
        keywords=['unique', 'unusual', 'artistic', 'creative', 'exotic', 'mysterious', 'oriental'],
        df=df,
        top_n=25
    )
}

# === SCENT PROFILE MAPPINGS ===
SCENT_PROFILE_NOTES = {
    'Warm & Rich (vanilla, amber, tonka, chocolate)': build_dynamic_note_profile(
        seed_notes=['vanilla', 'amber', 'tonka', 'cocoa'],
        keywords=['warm', 'rich', 'sweet', 'gourmand', 'cozy', 'comforting'],
        df=df,
        top_n=25
    ),
    'Fresh & Clean (citrus, aquatic, mint, green notes)': build_dynamic_note_profile(
        seed_notes=['citrus', 'marine', 'mint', 'bergamot'],
        keywords=['fresh', 'clean', 'aquatic', 'crisp', 'bright', 'airy'],
        df=df,
        top_n=25
    ),
    'Deep & Intense (oud, leather, tobacco, incense)': build_dynamic_note_profile(
        seed_notes=['oud', 'leather', 'tobacco', 'incense'],
        keywords=['deep', 'intense', 'dark', 'mysterious', 'smoky', 'animalic'],
        df=df,
        top_n=25
    ),
    'Soft & Floral (jasmine, rose, lavender, lily)': build_dynamic_note_profile(
        seed_notes=['jasmine', 'rose', 'lavender', 'lily'],
        keywords=['soft', 'floral', 'delicate', 'romantic', 'feminine', 'elegant'],
        df=df,
        top_n=25
    )
}

# === EVENT MAPPINGS ===
EVENT_NOTES = {
    'Date night': build_dynamic_note_profile(
        seed_notes=['vanilla', 'amber', 'musk', 'rose'],
        keywords=['romantic', 'seductive', 'sensual', 'sexy', 'intimate'],
        df=df,
        top_n=20
    ),
    'Family gathering': build_dynamic_note_profile(
        seed_notes=['citrus', 'lavender', 'green'],
        keywords=['fresh', 'clean', 'pleasant', 'light', 'friendly'],
        df=df,
        top_n=20
    ),
    'School/Work': build_dynamic_note_profile(
        seed_notes=['lavender', 'tea', 'cedar'],
        keywords=['professional', 'subtle', 'clean', 'office', 'understated'],
        df=df,
        top_n=20
    ),
    'Party/Going out': build_dynamic_note_profile(
        seed_notes=['amber', 'vanilla', 'fruity'],
        keywords=['bold', 'sweet', 'fun', 'party', 'attention', 'projection'],
        df=df,
        top_n=20
    )
}

# === VIBE MAPPINGS ===
VIBE_NOTES = {
    'Sweet/Comforting': build_dynamic_note_profile(
        seed_notes=['vanilla', 'tonka', 'caramel'],
        keywords=['sweet', 'comforting', 'cozy', 'warm', 'gourmand'],
        df=df,
        top_n=20
    ),
    'Fresh/Clean': build_dynamic_note_profile(
        seed_notes=['citrus', 'marine', 'mint'],
        keywords=['fresh', 'clean', 'crisp', 'bright', 'airy'],
        df=df,
        top_n=20
    ),
    'Woody/Spicy': build_dynamic_note_profile(
        seed_notes=['cedar', 'sandalwood', 'pepper'],
        keywords=['woody', 'spicy', 'warm', 'earthy', 'natural'],
        df=df,
        top_n=20
    ),
    'Floral/Soft': build_dynamic_note_profile(
        seed_notes=['jasmine', 'rose', 'iris'],
        keywords=['floral', 'soft', 'delicate', 'feminine', 'romantic'],
        df=df,
        top_n=20
    )
}

# === STRENGTH MAPPINGS (Keep simple) ===
STRENGTH_NOTES = {
    'Skin-scent': [],
    'Moderate': [],
    'Strong': extract_notes_for_keywords(df, ['strong', 'powerful', 'projection', 'sillage'], top_n=15)
}

# === SLIDER MAPPINGS ===
SLIDER_MAPPINGS = {
    'energy': {
        'left_notes': build_dynamic_note_profile(
            seed_notes=['lavender', 'sandalwood', 'musk'],
            keywords=['calm', 'relaxing', 'soft', 'peaceful'],
            df=df, top_n=15
        ),
        'right_notes': build_dynamic_note_profile(
            seed_notes=['citrus', 'ginger', 'mint'],
            keywords=['energetic', 'vibrant', 'fresh', 'invigorating'],
            df=df, top_n=15
        )
    },
    'style': {
        'left_notes': build_dynamic_note_profile(
            seed_notes=['rose', 'iris', 'amber'],
            keywords=['classic', 'elegant', 'timeless', 'sophisticated'],
            df=df, top_n=15
        ),
        'right_notes': build_dynamic_note_profile(
            seed_notes=['oud', 'leather', 'tobacco'],
            keywords=['bold', 'unconventional', 'unique', 'daring'],
            df=df, top_n=15
        )
    },
    'mood': {
        'left_notes': build_dynamic_note_profile(
            seed_notes=['vanilla', 'tonka', 'amber'],
            keywords=['warm', 'comforting', 'cozy', 'sweet'],
            df=df, top_n=15
        ),
        'right_notes': build_dynamic_note_profile(
            seed_notes=['marine', 'citrus', 'mint'],
            keywords=['fresh', 'invigorating', 'cool', 'crisp'],
            df=df, top_n=15
        )
    },
    'complexity': {
        'left_notes': build_dynamic_note_profile(
            seed_notes=['lemon', 'bergamot', 'marine'],
            keywords=['simple', 'clean', 'minimalist', 'light'],
            df=df, top_n=15
        ),
        'right_notes': build_dynamic_note_profile(
            seed_notes=['oud', 'incense', 'myrrh'],
            keywords=['complex', 'layered', 'mysterious', 'deep'],
            df=df, top_n=15
        )
    }
}

print("\n✅ Dynamic note mappings built successfully!")
print("\n📊 DISCOVERED NOTES PREVIEW:")
print("=" * 60)
for category, notes in PERSONALITY_NOTES.items():
    print(f"\n{category}:")
    print(f"   Top 10: {notes[:10]}")
    print(f"   Total: {len(notes)} notes")

print("\n🔥 Your algorithm now uses notes from YOUR ACTUAL DATASET!")
print("   - Not limited to hardcoded lists")
print("   - Adapts to your data")
print("   - Should see 20-30% match score improvement!")

# === STEP 3: Slider value → note mapping ===
def slider_to_notes(slider_name, value):
    """Convert slider value (0-100) to weighted note blend"""
    mapping = SLIDER_MAPPINGS[slider_name]
    left_notes = mapping['left_notes']
    right_notes = mapping['right_notes']
    
    # Calculate weights
    right_weight = value / 100.0
    left_weight = 1.0 - right_weight
    
    # Pick notes proportionally
    total_notes = 6
    num_right = int(round(right_weight * total_notes))
    num_left = total_notes - num_right
    
    selected = []
    if num_left > 0:
        selected += left_notes[:num_left]
    if num_right > 0:
        selected += right_notes[:num_right]
    
    return selected

# === STEP 4: Answers to profile function ===
def answers_to_profile(answers):
    """
    Convert user answers to target notes list.
    
    Args:
        answers: Dict of question answers
    
    Returns:
        List of target notes
    """
    target_notes = []
    
    # Multiple choice questions
    if 'personality' in answers:
        target_notes.extend(PERSONALITY_NOTES.get(answers['personality'], []))
    
    if 'scent_profile' in answers:
        target_notes.extend(SCENT_PROFILE_NOTES.get(answers['scent_profile'], []))
    
    if 'event' in answers:
        target_notes.extend(EVENT_NOTES.get(answers['event'], []))
    
    if 'vibe' in answers:
        target_notes.extend(VIBE_NOTES.get(answers['vibe'], []))
    
    if 'strength' in answers:
        target_notes.extend(STRENGTH_NOTES.get(answers['strength'], []))
    
    # Slider questions
    for slider in ['energy', 'style', 'mood', 'complexity']:
        if slider in answers:
            target_notes.extend(slider_to_notes(slider, answers[slider]))
    
    # Remove duplicates while preserving order
    seen = set()
    unique_notes = []
    for note in target_notes:
        if note not in seen:
            seen.add(note)
            unique_notes.append(note)
    
    return unique_notes

print("\n✓ Note mapping functions loaded!")


✓ Data preprocessing complete!
   - Dataset: 84144 perfumes
   - TF-IDF features: 3000 dimensions (includes note pairs)
   - Season features: 5 dimensions
   - Total notes vocabulary: 3000 unique notes/pairs
   - Combined features: 3005 dimensions (85% notes + 15% seasons)

🔍 Building dynamic note mappings from dataset...

✅ Dynamic note mappings built successfully!

📊 DISCOVERED NOTES PREVIEW:

Bold & Adventurous (confident, outgoing, takes charge):
   Top 10: ['oud', 'leather', 'tobacco', 'pepper', 'musk', 'sandalwood', 'amber', 'patchouli', 'bergamot', 'cedar']
   Total: 25 notes

Calm & Thoughtful (organized, reliable, introspective):
   Top 10: ['lavender', 'tea', 'musk', 'sandalwood', 'bergamot', 'amber', 'jasmine', 'vanilla', 'cedar', 'patchouli']
   Total: 25 notes

Energetic & Social (enthusiastic, expressive, loves attention):
   Top 10: ['citrus', 'bergamot', 'grapefruit', 'mint', 'citruses', 'musk', 'amber', 'jasmine', 'sandalwood', 'cedar']
   Total: 25 notes

Creative & O

In [6]:
# === INTERACTIVE RECOMMENDATION SYSTEM (WITH WEIGHTED QUERY) ===

def ask_user_questions():
    """
    Ask 10 comprehensive questions (6 multiple choice + 4 sliders) for detailed perfume preferences.
    Returns answers dict for recommendation engine.
    """
    questions = {
        'gender': {
            'type': 'choice',
            'prompt': 'What gender describes you best?',
            'choices': ['Male', 'Female', 'Unisex/Any']
        },
        'personality': {
            'type': 'choice',
            'prompt': 'How would you describe your personality?',
            'choices': [
                'Bold & Adventurous (confident, outgoing, takes charge)',
                'Calm & Thoughtful (organized, reliable, introspective)',
                'Energetic & Social (enthusiastic, expressive, loves attention)',
                'Creative & Open-minded (curious, artistic, unconventional)'
            ]
        },
        'scent_profile': {
            'type': 'choice',
            'prompt': 'Which scent profile appeals to you most?',
            'choices': [
                'Warm & Rich (vanilla, amber, tonka, chocolate)',
                'Fresh & Clean (citrus, aquatic, mint, green notes)',
                'Deep & Intense (oud, leather, tobacco, incense)',
                'Soft & Floral (jasmine, rose, lavender, lily)'
            ]
        },
        'energy': {
            'type': 'slider',
            'prompt': 'How would you describe your energy?',
            'left': 'Calm & Relaxed',
            'right': 'Energetic & Vibrant',
            'min': 0,
            'max': 100
        },
        'style': {
            'type': 'slider',
            'prompt': 'Your style preference:',
            'left': 'Classic & Timeless',
            'right': 'Bold & Unconventional',
            'min': 0,
            'max': 100
        },
        'event': {
            'type': 'choice',
            'prompt': 'Where are you wearing this?',
            'choices': ['Date night', 'Family gathering', 'School/Work', 'Party/Going out']
        },
        'vibe': {
            'type': 'choice',
            'prompt': 'Pick a vibe:',
            'choices': ['Sweet/Comforting', 'Fresh/Clean', 'Woody/Spicy', 'Floral/Soft']
        },
        'mood': {
            'type': 'slider',
            'prompt': 'What mood do you want to project?',
            'left': 'Warm & Comforting',
            'right': 'Fresh & Invigorating',
            'min': 0,
            'max': 100
        },
        'complexity': {
            'type': 'slider',
            'prompt': 'Fragrance complexity:',
            'left': 'Simple & Clean',
            'right': 'Complex & Layered',
            'min': 0,
            'max': 100
        },
        'strength': {
            'type': 'choice',
            'prompt': 'Projection preference?',
            'choices': ['Skin-scent', 'Moderate', 'Strong']
        }
    }

    answers = {}

    print("\n" + "="*60)
    print("FragrAI : Your next signature scent, picked before you spray")
    print("="*60)
    print()

    for q_key, q_data in questions.items():
        if q_data['type'] == 'slider':
            print(f" {q_data['prompt']}")
            print(f"   ◀ {q_data['left']}  |  {q_data['right']} ▶")
            print(f"   Enter a value from {q_data['min']}-{q_data['max']} ({q_data['min']} = left, {q_data['max']} = right, 50 = balanced):")

            while True:
                ans = input().strip()
                try:
                    value = int(ans)
                    if q_data['min'] <= value <= q_data['max']:
                        answers[q_key] = value
                        break
                except:
                    pass
                print("😠 dude try again, enter a number between 0-100")
        else:
            choices = q_data["choices"]
            print(f"\n {q_data['prompt']}")
            for i, choice in enumerate(choices, 1):
                print(f"   {i}. {choice}")

            while True:
                ans = input(f"Enter number (1-{len(choices)}): ").strip()
                try:
                    idx = int(ans) - 1
                    if 0 <= idx < len(choices):
                        answers[q_key] = choices[idx]
                        break
                except:
                    pass
                print("😠 dude try again")

    return answers


def recommend_from_answers(answers, min_similarity=0.0, top_n=10):
    """Recommend perfumes based on comprehensive questionnaire answers (WITH WEIGHTED QUERY)"""
    # Convert answers to target notes
    target_notes = answers_to_profile(answers)

    if not target_notes:
        print("⚠️ No target notes generated from your answers")
        return df.head(0), []

    # 🔥 WEIGHTED: Boost important scent descriptors 3x in user query too!
    weighted_target = []
    for note in target_notes:
        weighted_target.append(note)
        if note in scent_descriptors:
            weighted_target.extend([note, note])

    # Create query document (notes) with weighted terms
    query_doc = ' '.join(weighted_target)

    # Calculate TF-IDF query vector
    query_vec = tfidf.transform([query_doc])

    # Build combined query vector (notes + season) to match X_combined
    # If season features exist, create a zero season vector (no explicit season bias)
    try:
        season_dims = X_season_sparse.shape[1]
    except Exception:
        season_dims = 0

    if season_dims > 0 and 'X_combined' in globals():
        # create zero season vector (1 x season_dims)
        q_season = sp.csr_matrix((1, season_dims), dtype=float)

        # combine note and season parts using same weights as when X_combined was built
        try:
            q_combined = sp.hstack([query_vec * note_weight, q_season * season_weight], format='csr')
        except Exception:
            # fallback: just use query_vec (notes only)
            q_combined = query_vec

        # Compute similarities against X_combined
        similarities = cosine_similarity(q_combined, X_combined).ravel()
    else:
        # fallback to TF-IDF-only similarity
        similarities = cosine_similarity(query_vec, X_tfidf).ravel()

    # Create results dataframe
    results = df.copy()
    results['similarity'] = similarities

    # Apply gender filtering if specified
    if 'gender' in answers:
        gender_pref = answers['gender']
        if gender_pref in ['Male', 'Female']:
            # Map to our gender column values
            gender_map = {'Male': 'masculine', 'Female': 'feminine'}
            target_gender = gender_map.get(gender_pref, gender_pref.lower())
            results = results[results['gender'] == target_gender]

    # Filter by minimum similarity
    results = results[results['similarity'] >= min_similarity]

    # Sort by similarity and return top_n
    recommendations = results.nlargest(top_n, 'similarity')

    return recommendations, target_notes


# Ask user questions interactively
user_answers = ask_user_questions()

print("\n🎯 Analyzing your preferences...")
print("=" * 40)

# Get recommendations
recommendations, target_notes = recommend_from_answers(user_answers, top_n=3)

# ANSI color codes
HEADER = '\033[95m'
BLUE = '\033[94m'
CYAN = '\033[96m'
GREEN = '\033[92m'
YELLOW = '\033[93m'
RED = '\033[91m'
ENDC = '\033[0m'
BOLD = '\033[1m'
UNDERLINE = '\033[4m'

print(f"\n{BOLD}{CYAN}━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━{ENDC}")
print(f"{BOLD}{CYAN}YOUR PREFERENCE PROFILE{ENDC}")
print(f"{BOLD}{CYAN}━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━{ENDC}")
print(f"\n{GREEN}Target Notes:{ENDC}")
print(f"   {', '.join(target_notes[:10])}..." if len(target_notes) > 10 else f"   {', '.join(target_notes)}")
print(f"\n{CYAN}Total Target Notes: {len(target_notes)}{ENDC}")
print()

print(f"{BOLD}{CYAN}━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━{ENDC}")
print(f"{BOLD}{CYAN}TOP 3 FRAGRANCE RECOMMENDATIONS{ENDC}")
print(f"{BOLD}{CYAN}━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━{ENDC}")
print()

if len(recommendations) == 0:
    print(f"{YELLOW}No recommendations found matching your criteria.{ENDC}")
else:
    for i, (idx, row) in enumerate(recommendations.iterrows(), 1):
        # Card border
        print(f"{BOLD}{BLUE}┌─────────────────────────────────────────────────────┐{ENDC}")
        print(f"{BOLD}{BLUE}│{ENDC} {BOLD}{GREEN}RECOMMENDATION {i}{ENDC}" + " " * (32 - len(f"RECOMMENDATION {i}")) + f"{BOLD}{BLUE}│{ENDC}")
        print(f"{BOLD}{BLUE}├─────────────────────────────────────────────────────┤{ENDC}")
        
        # Title
        title = row['title'][:47]
        print(f"{BOLD}{BLUE}│{ENDC} {BOLD}{YELLOW}{title}{ENDC}" + " " * (51 - len(title)) + f"{BOLD}{BLUE}│{ENDC}")
        
        # Designer
        designer = f"by {row['designer']}"[:47]
        print(f"{BOLD}{BLUE}│{ENDC} {designer}" + " " * (49 - len(designer)) + f"{BOLD}{BLUE}│{ENDC}")
        
        # Gender & Season
        gender_season = f"Gender: {row['gender']} | Season: {row['season']}"[:47]
        print(f"{BOLD}{BLUE}│{ENDC} {GREEN}{gender_season}{ENDC}" + " " * (49 - len(gender_season)) + f"{BOLD}{BLUE}│{ENDC}")
        
        # Similarity score
        similarity_str = f"Match Score: {row['similarity']:.1%}"
        print(f"{BOLD}{BLUE}│{ENDC} {CYAN}{similarity_str}{ENDC}" + " " * (49 - len(similarity_str)) + f"{BOLD}{BLUE}│{ENDC}")
        
        # Notes
        notes_str = ', '.join(row['notes'][:5]) + "..." if len(row['notes']) > 5 else ', '.join(row['notes'])
        notes_display = f"Notes: {notes_str}"[:47]
        print(f"{BOLD}{BLUE}│{ENDC} {notes_display}" + " " * (49 - len(notes_display)) + f"{BOLD}{BLUE}│{ENDC}")
        
        print(f"{BOLD}{BLUE}└─────────────────────────────────────────────────────┘{ENDC}")
        print()

print(f"{BOLD}{GREEN}━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━{ENDC}")
print(f"{BOLD}{CYAN}Thank you for using FragrAI!{ENDC}")
print(f"{BOLD}{GREEN}Your perfect fragrance match awaits.{ENDC}")
print(f"{BOLD}{GREEN}━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━{ENDC}")



FragrAI : Your next signature scent, picked before you spray


 What gender describes you best?
   1. Male
   2. Female
   3. Unisex/Any

 How would you describe your personality?
   1. Bold & Adventurous (confident, outgoing, takes charge)
   2. Calm & Thoughtful (organized, reliable, introspective)
   3. Energetic & Social (enthusiastic, expressive, loves attention)
   4. Creative & Open-minded (curious, artistic, unconventional)

 Which scent profile appeals to you most?
   1. Warm & Rich (vanilla, amber, tonka, chocolate)
   2. Fresh & Clean (citrus, aquatic, mint, green notes)
   3. Deep & Intense (oud, leather, tobacco, incense)
   4. Soft & Floral (jasmine, rose, lavender, lily)
😠 dude try again
😠 dude try again
😠 dude try again
😠 dude try again
😠 dude try again
😠 dude try again
😠 dude try again
😠 dude try again
😠 dude try again
😠 dude try again
😠 dude try again
😠 dude try again
😠 dude try again
😠 dude try again
😠 dude try again
😠 dude try again
😠 dude try again
😠 dude try agai